<a href="https://colab.research.google.com/github/kaspar0852/Text-to-image-CLIP-and-Taming-transformer/blob/main/Text_To_Image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/openai/CLIP.git
!git clone https://github.com/CompVis/taming-transformers

In [ ]:
## install some libraries
!pip install --no-deps ftfy regex tqdm
!pip install omegaconf==2.0.0 pytorch-lightning==1.0.8
!pip uninstall torchtext --yes
!pip install einops

In [3]:
#import libraries
import numpy as np
import torch,os,imageio,pdb,math
import torchvision 
import torchvision.transforms as T
import torchvision.transforms.functional as TF

import PIL 
import matplotlib.pyplot as plt

import yaml
from omegaconf import OmegaConf

from CLIP import clip

#import warnings
#warnings.filterwarning('ignore')

In [4]:
##helper functions

def show_from_tensor(tensor):
  img = tensor.clone()
  img = img.mul(255).byte()
  img = img.cpu().numpy().transpose((1,2,0))

  plt.figure(figsize = (10,7))
  plt.axis('off')
  plt.imshow(img)
  plt.show()

  #this function will take tensor and display it as an image

def norm_data(data):
   #it will receive the data and normalize it in a follwoing way
   return(data.clip(-1,1)+1)/2 ##we are going to move in between 0 to 1 in result

##Parameters and Hyperparameters
learning_rate = .5
batch_size = 1
wd = .1 #weight decay is a regulizer paramerter that is going to help the optimizer limit the size-
#-of weights to improve the generalization capabilities of the architecture
noise_factor = .1 

total_iter = 100 #increace this for more refined results
#as we take a text prompt and we begin to generate the image,then there will be comparison of encoding of text prompt-
#-with the enoding of the image we are generating and we calculate the loss suing the Cosine Similarity-
#-mathematical function applied to the result of those encodings

im_shape = [225,400,3] #height,width ,channel
size1, size2,channels = im_shape


In [5]:
#Setting up the CLIP model
clipmodel, _ = clip.load('ViT-B/32',jit = False)
clipmodel.eval()
print(clip.available_models())

print('Clip model visual input resolution:',clipmodel.visual.input_resolution)

device = torch.device("cuda:0")
torch.cuda.empty_cache()
#We can proceed to use clip in inferrence, eval mode to encode texts and images directly

100%|████████████████████████████████████████| 338M/338M [00:03<00:00, 102MiB/s]


['RN50', 'RN101', 'RN50x4', 'RN50x16', 'RN50x64', 'ViT-B/32', 'ViT-B/16', 'ViT-L/14', 'ViT-L/14@336px']
Clip model visual input resolution: 224


In [ ]:
## Taming transformer instantiation

%cd taming-transformers/

!mkdir -p models/vqgan_imagenet_f16_16384/checkpoints
!mkdir -p models/vqgan_imagenet_f16_16384/configs

if len(os.listdir('models/vqgan_imagenet_f16_16384/checkpoints/')) == 0:
   !wget 'https://heibox.uni-heidelberg.de/f/867b05fc8c4841768640/?dl=1' -O 'models/vqgan_imagenet_f16_16384/checkpoints/last.ckpt' 
   !wget 'https://heibox.uni-heidelberg.de/f/274fb24ed38341bfa753/?dl=1' -O 'models/vqgan_imagenet_f16_16384/configs/model.yaml' 


In [ ]:
from taming.models.vqgan import VQModel

#we need a couple of helper functions
def load_config(config_path,display = False):
  config_data = OmegaConf.load(config_path)
  if display:
    print(yaml.dump(OmegaConf.to_container(config_data)))
  return config_data

def load_vqgan(config,chk_path = None):
  model = VQModel(**config.model.params)
  if chk_path is not None:
    state_dict = torch.load(chk_path, map_location ="cpu")["state_dict"]
    missing,unexpected = model.load_state_dict(state_dict,strict = False)
  return model.eval()

def generator(x):
  x = taming_model.post_quant_conv(x)
  x = taming_model.decoder(x)
  return x 
  #when we generate something from this model,it has to go through two stages-
  #-1 is the convolutional network and 2nd actual decoder of the transformer

taming_config = load_config("./models/vqgan_imagenet_f16_16384/configs/model.yaml",display = True)
taming_model = load_vqgan(taming_config,chk_path = "./models/vqgan_imagenet_f16_16384/checkpoints/last.ckpt").to(device)


In [9]:
#we are seeing all the detailed information of the generator /transformer model when we execute the -
#-top code it uses 256 channels,and patches of 16 * 16 pixels each